# Stage 1 - Make and train the model

In [ ]:
import pandas as pd
import numpy as np
import torch

## dataset generation
# lets say we want to train a function that returns max(x,y)

input_data = pd.DataFrame({
    'x': np.random.uniform(-1000, 1000, 10000),
    'y': np.random.uniform(-1000, 1000, 10000)
})
output_data = pd.DataFrame({
    'output': np.maximum(input_data['x'], input_data['y'])
})


df = pd.concat([input_data, output_data], axis=1)

df.describe()

,x,y,max
count,10000.000000,10000.000000,10000.000000
mean,-3.319804,3.377585,336.459274
std,577.131497,573.395977,466.857149
min,-999.540014,-999.991621,-970.509688
25%,-502.060301,-485.131598,7.165847
50%,-7.168021,4.138413,414.690881
75%,497.127856,497.560860,730.794642
max,999.305530,999.432485,999.432485


In [ ]:
# model definition and training

class SimpleModel(torch.nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.linear1 = torch.nn.Linear(2, 10)
        self.linear2 = torch.nn.Linear(10, 1)

    def forward(self, x):
        x = torch.relu(self.linear1(x))
        x = self.linear2(x)
        return x
    
    # this method is necessary for the model to be evaluated by poly-lithic

    def evaluate(self, x):
        # x will be a dicrt of keys and values
        # {"x": x, "y": y}
        input_tensor = torch.tensor([x['x'], x['y']], dtype=torch.float32)
        # you may want to do somethinf more complex here
        output_tensor = self.forward(input_tensor)
        # return a dictionary of keys and values
        return {"output": output_tensor.item()}        
    
# model training
model = SimpleModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.MSELoss()
batch_size = 32

# convert data to torch tensors
X = torch.tensor(df[['x', 'y']].values, dtype=torch.float32)
y = torch.tensor(df[['output']].values, dtype=torch.float32)
# training loop
for epoch in range(100):
    for i in range(0, len(X), batch_size):
        X_batch = X[i:i+batch_size]
        y_batch = y[i:i+batch_size]

        # forward pass
        y_pred = model(X_batch)

        # compute loss
        loss = loss_fn(y_pred, y_batch)

        # backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch}, Loss: {loss.item()}')
    
# save the model
torch.save(model.state_dict(), 'local/model.pth')
    

Epoch 0, Loss: 3.027113676071167
Epoch 1, Loss: 0.6868435144424438
Epoch 2, Loss: 0.5107424855232239
Epoch 3, Loss: 0.5535025596618652
Epoch 4, Loss: 0.5595170855522156
Epoch 5, Loss: 0.45030251145362854
Epoch 6, Loss: 0.37930625677108765
Epoch 7, Loss: 0.5606532096862793
Epoch 8, Loss: 0.7571465969085693
Epoch 9, Loss: 0.363572359085083
Epoch 10, Loss: 0.510280966758728
Epoch 11, Loss: 0.3281683921813965
Epoch 12, Loss: 0.21552875638008118
Epoch 13, Loss: 0.264011949300766
Epoch 14, Loss: 0.39230236411094666
Epoch 15, Loss: 1.6711008548736572
Epoch 16, Loss: 0.7677030563354492
Epoch 17, Loss: 0.10186141729354858
Epoch 18, Loss: 1.390590786933899
Epoch 19, Loss: 1.641930103302002
Epoch 20, Loss: 0.006918319500982761
Epoch 21, Loss: 0.29741910099983215
Epoch 22, Loss: 9.805606842041016
Epoch 23, Loss: 0.05324534326791763
Epoch 24, Loss: 0.2600162923336029
Epoch 25, Loss: 0.03549977019429207
Epoch 26, Loss: 4.396921157836914
Epoch 27, Loss: 0.047415345907211304
Epoch 28, Loss: 0.06699024

In [5]:
# test the model
test_data = pd.DataFrame({
    'x': np.random.uniform(-1000, 1000, 100),
    'y': np.random.uniform(-1000, 1000, 100)
})
test_X = torch.tensor(test_data[['x', 'y']].values, dtype=torch.float32)
test_y = torch.tensor(np.maximum(test_data['x'], test_data['y']), dtype=torch.float32)
test_y_pred = model(test_X)
test_loss = loss_fn(test_y_pred, test_y)
print(f'Test Loss: {test_loss.item()/len(test_X)}')

# print sample
for i in range(10):
    print(f"Input: {test_data.iloc[i].values}, Predicted: {test_y_pred[i].item()}, Actual: {test_y[i].item()}")

Test Loss: 4179.904375
Input: [ 573.66538305 -497.08576156], Predicted: 573.4780883789062, Actual: 573.6654052734375
Input: [-755.38952928  165.59900853], Predicted: 165.28440856933594, Actual: 165.59901428222656
Input: [-728.08504207  197.48211559], Predicted: 197.1942901611328, Actual: 197.48211669921875
Input: [355.55092175 976.00125353], Predicted: 975.845703125, Actual: 976.0012817382812
Input: [-386.99307363 -953.92977677], Predicted: -387.9506530761719, Actual: -386.9930725097656
Input: [ 562.55284657 -884.40475181], Predicted: 562.03857421875, Actual: 562.5528564453125
Input: [-733.48783562 -964.21029821], Predicted: -734.5945434570312, Actual: -733.4878540039062
Input: [418.205749   -30.50748379], Predicted: 418.2569580078125, Actual: 418.20574951171875
Input: [254.79010163 291.42216212], Predicted: 291.3809509277344, Actual: 291.4221496582031
Input: [-934.28593574  474.82438214], Predicted: 474.5726013183594, Actual: 474.8243713378906


/home/gbm96348/nfs_home/ml_env/lib/python3.11/site-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


# Stage 2a - Write a poly-lithic compatible model  - from local file system

Look inside `examples/base/local/model_definition.py`

The file is composed of a model factory and a model definition. Which we point to in the deployment.yaml file.


Model definition as beofre:

``` python
class SimpleModel(torch.nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.linear1 = torch.nn.Linear(2, 10)
        self.linear2 = torch.nn.Linear(10, 1)

    def forward(self, x):
        x = torch.relu(self.linear1(x))
        x = self.linear2(x)
        return x
    
    # this method is necessary for the model to be evaluated by poly-lithic
    
    def evaluate(self, x: dict) -> dict:
        # x will be a dicrt of keys and values
        # {"x": x, "y": y}
        input_tensor = torch.tensor([x['x'], x['y']], dtype=torch.float32)
        # you may want to do somethinf more complex here
        output_tensor = self.forward(input_tensor)
        # return a dictionary of keys and values
        return {"output": output_tensor.item()}
```

Model factory:

``` python
class ModelFactory:
    
    # can do more complex things here but we will just load the model from a locally saved file
    def __init__(self):
        self.model = SimpleModel()
        self.model = self.model.load_state_dict(torch.load('model.pth'))
        # other magic init stuff here
        print("ModelFactory initialized")
    
    # this method is necessary for the model to be retrieved by poly-lithic
    def get_model(self):
        return self.model
```



In [16]:
# lets check our import works
from local.model_definition import ModelFactory

factory = ModelFactory()

model = factory.get_model()


sample_input = {
    "x": -20,
    "y": 1000.0
}
sample_output = model.evaluate(sample_input)
print(f"Input: {sample_input}, Output: {sample_output}")


Model loaded successfully.
ModelFactory initialized
Input: {'x': -20, 'y': 1000.0}, Output: {'max': 999.8095092773438}


In this case our `deployment_config.yaml` file will look like this:

``` yaml
deployment:
  type: "continuous"
  # other configurations
input_data:
  get_method: "p4p_server"
  config:
    # EPICS_PVA_NAME_SERVERS: "localhost:5075"
    # intialize: false
    variables:
      LUME:MLFLOW:TEST_B:
        proto: pva
        name: LUME:MLFLOW:TEST_B
      LUME:MLFLOW:TEST_A:
        proto: pva
        name: LUME:MLFLOW:TEST_A

input_data_to_model:
  type: "SimpleTransformer"
  config:
    symbols:
      - "LUME:MLFLOW:TEST_B"
      - "LUME:MLFLOW:TEST_A"
    variables:
      x2:
        formula: "LUME:MLFLOW:TEST_B"
      x1: 
        formula: "LUME:MLFLOW:TEST_A"

outputs_model:
  config:
    variables:
      y:
        type: "scalar"

output_model_to_data:
  type: "SimpleTransformer"
  config:
    symbols:
      - "y"
    variables:
      LUME:MLFLOW:TEST_G:
        formula: "y"

output_data_to:
  put_method: "p4p_server"
  config:
    variables:
      LUME:MLFLOW:TEST_G:
        proto: pva
        name: LUME:MLFLOW:TEST_G


# Stage 2b - Write a poly-lithic compatible model  - from mlflow